<img align="center" style="max-width: 1000px" src="../assets/banner.png?raw=1">

<img align="right" style="max-width: 200px; height: auto" src="../assets/hsg_logo.png?raw=1">

##  Lab 04 - Artificial Neural Networks (ANNs) - Assignments

EMBA: Coding und Künstliche Intelligenz, University of St. Gallen

In the last lab we learned how to implement, train, and apply our first **Artificial Neural Network (ANN)** using a Python library named `PyTorch`. The `PyTorch` library is an open-source machine learning library for Python, used for a variety of applications such as image classification and natural language processing. In this lab, we aim to leverage that knowledge by applying it to a set of self-coding assignments.

As always, please don't hesitate to ask all your questions either during the lab, post them in our CANVAS (StudyNet) forum (https://learning.unisg.ch), or send us an email (using the course email).

## 1. Assignment Objectives:

Similar today's lab session, after today's self-coding assignments you should be able to:

> 1. Understand the basic concepts, intuitions and major building blocks of **Artificial Neural Networks (ANNs)**.
> 2. Know how to use Python's **PyTorch library** to train and evaluate neural network based models.
> 3. Understand how to apply neural networks to **classify images** of handwritten digits.
> 4. Know how to **interpret the detection results** of the network as well as its **reconstruction loss**.

## 2. Setup of the Jupyter Notebook Environment

Similar to the previous labs, we need to import a couple of Python libraries that allow for data analysis and data visualization. We will mostly use the `PyTorch`, `Numpy`, `Sklearn`, `Matplotlib`, `Seaborn` and a few utility libraries throughout this lab:

In [ ]:
# import standard python libraries
import os, urllib, io
from datetime import datetime
import numpy as np

Import the Python machine / deep learning libraries:

In [ ]:
# import the PyTorch deep learning libary
import torch, torchvision
import torch.nn.functional as F
from torch import nn, optim

Import the sklearn classification metrics:

In [ ]:
# import sklearn classification evaluation library
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

Import Python plotting libraries:

In [ ]:
# import matplotlib, seaborn, and PIL data visualization libary
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

Enable notebook matplotlib inline plotting:

In [ ]:
%matplotlib inline

Import Google's GDrive connector and mount your GDrive directories:

In [ ]:
# import the Google Colab GDrive connector
from google.colab import drive

# mount GDrive inside the Colab notebook
drive.mount('/content/drive')

Create a structure of Colab Notebook sub-directories inside of GDrive to store (1) the data as well as (2) the trained neural network models:

In [ ]:
# create Colab Notebooks directory
notebook_directory = '/content/drive/MyDrive/Colab Notebooks'
if not os.path.exists(notebook_directory): os.makedirs(notebook_directory)

 # create data sub-directory inside the Colab Notebooks directory
data_directory = '/content/drive/MyDrive/Colab Notebooks/data'
if not os.path.exists(data_directory): os.makedirs(data_directory)

 # create models sub-directory inside the Colab Notebooks directory
models_directory = '/content/drive/MyDrive/Colab Notebooks/models'
if not os.path.exists(models_directory): os.makedirs(models_directory)

Set a random `seed` value to obtain reproducable results:

In [ ]:
# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

Google Colab provides the use of free GPUs for running notebooks. However, if you just execute this notebook as is, it will use your device's CPU. To run the lab on a GPU, got to `Runtime` > `Change runtime type` and set the Runtime type to `GPU` in the drop-down. Running this lab on a CPU is fine, but you will find that GPU computing is faster. *CUDA* indicates that the lab is being run on GPU.

Enable GPU computing by setting the device flag and init a CUDA seed:

In [ ]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

Let's determine if we have access to a GPU provided by e.g. Google's COLab environment:

In [ ]:
!nvidia-smi

## 3. Artifcial Neural Networks (ANNs) Assignments

### 3.1 Fashion MNIST Dataset Download and Data Assessment

The **Fashion-MNIST database** is a large database of Zalando articles that is commonly used for training various image processing systems. The database is widely used for training and testing in the field of machine learning. Let's have a brief look into a couple of sample images contained in the dataset:

<img align="center" style="max-width: 700px; height: 300px" src="../lab_04/FashionMNIST.png">

Source: https://www.kaggle.com/c/insar-fashion-mnist-challenge

Further details on the dataset can be obtained via Zalando research's [github page](https://github.com/zalandoresearch/fashion-mnist).

The **Fashion-MNIST database** is a dataset of Zalando's article images, consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando created this dataset with the intention of providing a replacement for the popular **MNIST** handwritten digits dataset. It is a useful addition as it is a bit more complex, but still very easy to use. It shares the same image size and train/test split structure as MNIST, and can therefore be used as a drop-in replacement. It requires minimal efforts on preprocessing and formatting the distinct images.

Let's download, transform and inspect the training images of the dataset. Therefore, let's first define the directory in which we aim to store the training data:

In [ ]:
train_path = data_directory + '/train_fashion_mnist'

Now, let's download the training data accordingly:

In [ ]:
# define pytorch transformation into tensor format
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

# download and transform training images
fashion_mnist_train_data = torchvision.datasets.FashionMNIST(root=train_path, train=True, transform=transf, download=True)

Verify the number of training images downloaded:

In [ ]:
# determine the number of training data images
len(fashion_mnist_train_data)

Next, we need to map each numerical label to its fashion item, which will be useful throughout the lab:

In [ ]:
fashion_classes = {0: 'T-shirt/top',
                    1: 'Trouser',
                    2: 'Pullover',
                    3: 'Dress',
                    4: 'Coat',
                    5: 'Sandal',
                    6: 'Shirt',
                    7: 'Sneaker',
                    8: 'Bag',
                    9: 'Ankle boot'}

Let's now define the directory in which we aim to store the evaluation data:

In [ ]:
eval_path = data_directory + '/eval_fashion_mnist'

And download the evaluation data accordingly:

In [ ]:
# define pytorch transformation into tensor format
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

# download and transform training images
fashion_mnist_eval_data = torchvision.datasets.FashionMNIST(root=eval_path, train=False, transform=transf, download=True)

Let's also verify the number of evaluation images downloaded:

In [ ]:
# determine the number of evaluation data images
len(fashion_mnist_eval_data)

### 3.2 Artificial Neural Network (ANN) Model Training and Evaluation

<img align="center" style="max-width: 1000px" src="../lab_04/process.png">

We recommend you to try the following exercises as part of the self-coding session:

**Exercise 1: Train the neural network architecture of the lab for less epochs and evaluate its prediction accuracy.**

> Decrease the number of training epochs to **5 epochs** and re-run the network training. Load and evaluate the model exhibiting the lowest training loss. What kind of behaviour in terms of prediction accuracy can be observed with decreasing the number of training epochs?

In [ ]:
# ***************************************************
# INSERT YOUR SOLUTION/CODE HERE
# ***************************************************

# ***************************************************
# Task 1: define and init neural network architecture
# ***************************************************

# implement the MNISTNet network architecture
class FashionMNISTNet(nn.Module):
    
    # define the class constructor
    def __init__(self):
        
        # ***************************************************
        # insert the network architecture here
        # ***************************************************
        
    # define network forward pass
    def forward(self, images):
        
        # ***************************************************
        # insert the network forwad pass here
        # ***************************************************
        
        # return forward pass result
        return x


# init the neural network model
model = ???

# put the model on GPU
???

# ***************************************************
# Task 2: define loss, training hyperparameters and dataloader
# ***************************************************

# define the optimization criterion / loss function
nll_loss = ???

# specify the training parameters
num_epochs = ??? # number of training epochs
mini_batch_size = ??? # size of the mini-batches

# define learning rate and optimization strategy
learning_rate = ??? 
optimizer = optim.SGD(params=model.parameters(), lr=learning_rate)

# init the training data loader
fashion_mnist_train_dataloader = torch.utils.data.DataLoader(fashion_mnist_train_data, batch_size=mini_batch_size, shuffle=True)

# ***************************************************
# Task 3: run model training
# ***************************************************

# init collection of training epoch losses
train_epoch_losses = []

# set the model in training mode
model.train()

# train the MNISTNet model
for epoch in range(num_epochs):
    
    # init collection of mini-batch losses
    train_mini_batch_losses = []
    
    # iterate over all-mini batches
    for i, (images, labels) in enumerate(fashion_mnist_train_dataloader):
        # put the mini-batch data on GPU
        images = ???
        labels = ???

        # run forward pass through the network
        output = ???
        
        # reset graph gradients
        model.zero_grad()
        
        # determine classification loss
        loss = ???
        
        # run backward pass
        loss.backward()
        
        # update network paramaters
        optimizer.step()
        
        # collect mini-batch reconstruction loss
        train_mini_batch_losses.append(loss.data.item())
    
    # determine mean min-batch loss of epoch
    train_epoch_loss = np.mean(train_mini_batch_losses)
    
    # print epoch loss
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))

    # determine mean min-batch loss of epoch
    train_epoch_losses.append(train_epoch_loss)

# ***************************************************
# Task 4: run model evaluation
# ***************************************************

# determine model predictions
predictions = torch.argmax(model(???, dim=1)

# determine accuracy scores
accuracy = metrics.accuracy_score(???, ???)
                           
# print the classification accuracy percentage
print('Final FashionMNISTNet classification accuracy: {}%'.format(accuracy * 100))

**Exercise 2: Evaluation of "shallow" vs. "deep" neural network architectures.**

> In addition to the architecture of the lab notebook, evaluate further (more **shallow** as well as more **deep**) neural network architectures by (1) either **removing or adding** layers to the network and/or (2) increasing/decreasing the number of neurons per layer. Train a model (using the architectures you selected) for at least **20 training epochs**. Analyze the prediction performance of the trained models in terms of training time and prediction accuracy. 

In [ ]:
# ***************************************************
# INSERT YOUR SOLUTION/CODE HERE
# ***************************************************

# ***************************************************
# Task 1: define and init neural network architecture
# ***************************************************

# implement the MNISTNet network architecture
class FashionMNISTNet(nn.Module):
    
    # define the class constructor
    def __init__(self):
        
        # ***************************************************
        # insert the network architecture here
        # ***************************************************
        
    # define network forward pass
    def forward(self, images):
        
        # ***************************************************
        # insert the network forwad pass here
        # ***************************************************
        
        # return forward pass result
        return x


# init the neural network model
model = ???

# put the model on GPU
???

# ***************************************************
# Task 2: define loss, training hyperparameters and dataloader
# ***************************************************

# define the optimization criterion / loss function
nll_loss = ???

# specify the training parameters
num_epochs = ??? # number of training epochs
mini_batch_size = ??? # size of the mini-batches

# define learning rate and optimization strategy
learning_rate = ??? 
optimizer = optim.SGD(params=model.parameters(), lr=learning_rate)

# init the training data loader
fashion_mnist_train_dataloader = torch.utils.data.DataLoader(fashion_mnist_train_data, batch_size=mini_batch_size, shuffle=True)

# ***************************************************
# Task 3: run model training
# ***************************************************

# init collection of training epoch losses
train_epoch_losses = []

# set the model in training mode
model.train()

# train the MNISTNet model
for epoch in range(num_epochs):
    
    # init collection of mini-batch losses
    train_mini_batch_losses = []
    
    # iterate over all-mini batches
    for i, (images, labels) in enumerate(fashion_mnist_train_dataloader):
        # put the mini-batch data on GPU
        images = ???
        labels = ???

        # run forward pass through the network
        output = ???
        
        # reset graph gradients
        model.zero_grad()
        
        # determine classification loss
        loss = ???
        
        # run backward pass
        loss.backward()
        
        # update network paramaters
        optimizer.step()
        
        # collect mini-batch reconstruction loss
        train_mini_batch_losses.append(loss.data.item())
    
    # determine mean min-batch loss of epoch
    train_epoch_loss = np.mean(train_mini_batch_losses)
    
    # print epoch loss
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))

    # determine mean min-batch loss of epoch
    train_epoch_losses.append(train_epoch_loss)

# ***************************************************
# Task 4: run model evaluation
# ***************************************************

# determine model predictions
predictions = torch.argmax(model(???, dim=1)

# determine accuracy scores
accuracy = metrics.accuracy_score(???, ???)
                           
# print the classification accuracy percentage
print('Final FashionMNISTNet classification accuracy: {}%'.format(accuracy * 100))